In [23]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import SparseRandomProjection, johnson_lindenstrauss_min_dim
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, matthews_corrcoef
from itertools import product

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Data Extraction and Vectorization

# Parameters setup
flakyZip = "Dataset/flaky_files.zip"
largerNonFlakyZip = "Dataset/all_nonflaky_files.zip"

# Create directories
outDirLarger = "results/larger_nonflaky/"
os.makedirs(outDirLarger, exist_ok=True)

extractDirLarger = "extracted/larger_nonflaky/"
os.makedirs(extractDirLarger, exist_ok=True)

# Extract and read data once for equal combination
os.makedirs(nonFlakyDirEqual, exist_ok=True)




# Vectorize data once

# Extract and read data once for larger non-flaky combination
flakyDirLarger = os.path.join(extractDirLarger, 'flaky')
nonFlakyDirLarger = os.path.join(extractDirLarger, 'nonFlaky')
os.makedirs(flakyDirLarger, exist_ok=True)
os.makedirs(nonFlakyDirLarger, exist_ok=True)

extract_zip(flakyZip, flakyDirLarger)

dataPointsFlakyLarger = getDataPoints(flakyDirLarger)
dataPointsNonFlakyLarger = getDataPoints(nonFlakyDirLarger)
dataPointsLarger = dataPointsFlakyLarger + dataPointsNonFlakyLarger

# Print the number of datasets for larger combination
print(f"Number of flaky documents (larger combination): {len(dataPointsFlakyLarger)}")
print(f"Number of non-flaky documents (larger combination): {len(dataPointsNonFlakyLarger)}")
print(f"Total number of documents (larger combination): {len(dataPointsLarger)}")

dataLabelsListLarger = np.array([1]*len(dataPointsFlakyLarger) + [0]*len(dataPointsNonFlakyLarger))

Z_larger = flastVectorization(dataPointsLarger, dim=100, eps=0.3)


Number of flaky documents (larger combination): 45
Number of non-flaky documents (larger combination): 243
Total number of documents (larger combination): 288


## Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.random_projection import SparseRandomProjection

from sklearn.random_projection import johnson_lindenstrauss_min_dim



def flastRFWithThresholds(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization without PCA
    Z = flastVectorization(dataPoints)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")

        # Define the pipeline
        pipeline = Pipeline([
            ('rf', RandomForestClassifier(
                n_estimators=param_dict['rf__n_estimators'],
                criterion=param_dict['rf__criterion'],
                max_depth=param_dict['rf__max_depth'],
                min_samples_split=param_dict['rf__min_samples_split'],
                min_samples_leaf=param_dict['rf__min_samples_leaf'],
                random_state=42,
                n_jobs=-1
            )),
        ])

        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Convert to dense format for PCA
            X_train_dense = X_train.toarray()
            X_test_dense = X_test.toarray()

            # Train the model
            pipeline.fit(X_train_dense, y_train)

            # Predict probabilities on test set
            y_pred_proba = pipeline.predict_proba(X_test_dense)

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'rf__n_estimators': param_dict['rf__n_estimators'],
                    'rf__criterion': param_dict['rf__criterion'],
                    'rf__max_depth': param_dict['rf__max_depth'],
                    'rf__min_samples_split': param_dict['rf__min_samples_split'],
                    'rf__min_samples_leaf': param_dict['rf__min_samples_leaf'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return None, None

    # Convert the list of metrics into a DataFrame
    df_results = pd.DataFrame(metrics_per_combination)

    # Save the per-fold results
    outFile = f"{combination_label}-thresholds-rf-results-per-fold.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Per-fold Random Forest analysis completed. Results saved to: {outFile}")

    # Compute the average metrics across folds for each combination of hyperparameters and thresholds
    grouped_metrics = df_results.groupby(
        [ 'rf__n_estimators', 'rf__criterion', 'rf__max_depth', 'rf__min_samples_split', 'rf__min_samples_leaf', 'threshold']
    ).agg({
        'accuracy': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'mcc': ['mean', 'std']
    }).reset_index()

    # Flatten the MultiIndex columns
    grouped_metrics.columns = [
        '_'.join(col).strip('_') if col[1] else col[0] 
        for col in grouped_metrics.columns.values
    ]

    # Save the averaged results
    outFileAvg = f"{combination_label}-thresholds-rf-results-averaged.csv"
    grouped_metrics.to_csv(os.path.join(outDir, outFileAvg), index=False)

    print(f"Averaged results saved to: {outFileAvg}")

    return df_results, grouped_metrics

if __name__ == "__main__":
    #parameter grid
    param_grid = {
    'rf__n_estimators': [50, 100, 200],                   # Number of trees in the forest
    'rf__max_depth': [10, 20, 30],                        # Maximum depth of the tree
    'rf__min_samples_split': [5, 10],                     # Minimum number of samples required to split a node
    'rf__min_samples_leaf': [2, 5],                       # Minimum number of samples required at a leaf node
    'rf__criterion': ['gini', 'entropy']        
    }

    # Paths to your datasets
    flakyZip = "Dataset/flaky_files.zip"
    nonFlakyZip = "Dataset/nonflaky_files.zip"  

    # Create result and extract directories
    outDir = "results/rf_thresholds/"
    os.makedirs(outDir, exist_ok=True)
    extractDir = "extracted/rf_thresholds/"
    os.makedirs(extractDir, exist_ok=True)

    # Perform Random Forest analysis with threshold adjustments
    print("Starting Random Forest analysis with threshold adjustments...")
    df_results, grouped_metrics = flastRFWithThresholds(
        outDir, flakyZip, nonFlakyZip, extractDir, n_splits=5,
        combination_label="rf_thresholds", param_grid=param_grid)

    if df_results is not None:
        print("Analysis completed. Per-fold Results:")
        print(df_results.head())

        print("Averaged Results:")
        print(grouped_metrics.head())
    else:
        print("Analysis did not produce any results.")


Starting Random Forest analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 243
Total number of documents: 288
Training with parameters: {'rf__n_estimators': 50, 'rf__max_depth': 10, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 2, 'rf__criterion': 'gini'}
Training with parameters: {'rf__n_estimators': 50, 'rf__max_depth': 10, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 2, 'rf__criterion': 'entropy'}
Training with parameters: {'rf__n_estimators': 50, 'rf__max_depth': 10, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 5, 'rf__criterion': 'gini'}
Training with parameters: {'rf__n_estimators': 50, 'rf__max_depth': 10, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 5, 'rf__criterion': 'entropy'}
Training with parameters: {'rf__n_estimators': 50, 'rf__max_depth': 10, 'rf__min_samples_split': 10, 'rf__min_samples_leaf': 2, 'rf__criterion': 'gini'}
Training with parameters: {'rf__n_estimators': 50, 'rf__max_depth': 10, 'rf

Training with parameters: {'rf__n_estimators': 200, 'rf__max_depth': 10, 'rf__min_samples_split': 10, 'rf__min_samples_leaf': 2, 'rf__criterion': 'entropy'}
Training with parameters: {'rf__n_estimators': 200, 'rf__max_depth': 10, 'rf__min_samples_split': 10, 'rf__min_samples_leaf': 5, 'rf__criterion': 'gini'}
Training with parameters: {'rf__n_estimators': 200, 'rf__max_depth': 10, 'rf__min_samples_split': 10, 'rf__min_samples_leaf': 5, 'rf__criterion': 'entropy'}
Training with parameters: {'rf__n_estimators': 200, 'rf__max_depth': 20, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 2, 'rf__criterion': 'gini'}
Training with parameters: {'rf__n_estimators': 200, 'rf__max_depth': 20, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 2, 'rf__criterion': 'entropy'}
Training with parameters: {'rf__n_estimators': 200, 'rf__max_depth': 20, 'rf__min_samples_split': 5, 'rf__min_samples_leaf': 5, 'rf__criterion': 'gini'}
Training with parameters: {'rf__n_estimators': 200, 'rf__max_depth': 2

## Decision Tree

In [25]:
from sklearn.tree import DecisionTreeClassifier

def flastDTWithThresholds(outDir, flakyZip, nonFlakyZip, extractDir, n_splits, combination_label, param_grid):
    v0 = time.perf_counter()

    # Extract the zip files
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    os.makedirs(flakyDir, exist_ok=True)
    os.makedirs(nonFlakyDir, exist_ok=True)
    
    extract_zip(flakyZip, flakyDir)
    extract_zip(nonFlakyZip, nonFlakyDir)

    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky

    print(f"Number of flaky documents: {len(dataPointsFlaky)}")
    print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
    print(f"Total number of documents: {len(dataPoints)}")
    
    if len(dataPoints) == 0:
        raise ValueError("No documents available for vectorization. Please check the input directories.")

    # Vectorization without PCA
    Z = flastVectorization(dataPoints)
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    vecTime = time.perf_counter() - v0

    # Define StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Initialize storage for metrics per fold and threshold
    thresholds = np.linspace(0.1, 0.9, 9)
    metrics_per_combination = []

    # Manually iterate over all combinations of hyperparameters
    for params in product(*param_grid.values()):
        # Convert params from tuple to dictionary
        param_dict = dict(zip(param_grid.keys(), params))
        print(f"Training with parameters: {param_dict}")

        # Define the pipeline with PCA and Decision Tree
        pipeline = Pipeline([
            ('dt', DecisionTreeClassifier(
                criterion=param_dict['dt__criterion'],
                max_depth=param_dict['dt__max_depth'],
                min_samples_split=param_dict['dt__min_samples_split'],
                min_samples_leaf=param_dict['dt__min_samples_leaf'],
                max_features=param_dict['dt__max_features'],
                random_state=42,
            )),
        ])

        # Cross-validation
        for fold, (train_index, test_index) in enumerate(skf.split(Z, dataLabelsList)):
            X_train, X_test = Z[train_index], Z[test_index]
            y_train, y_test = dataLabelsList[train_index], dataLabelsList[test_index]

            if sum(y_train) == 0 or sum(y_test) == 0:
                print(f"Skipping fold {fold+1} due to no positive samples in train or test set")
                continue

            # Convert to dense format for PCA
            X_train_dense = X_train.toarray()
            X_test_dense = X_test.toarray()

            # Train the model
            pipeline.fit(X_train_dense, y_train)

            # Predict probabilities on test set
            if hasattr(pipeline.named_steps['dt'], "predict_proba"):
                y_pred_proba = pipeline.predict_proba(X_test_dense)
            else:
                # If predict_proba is not available, use decision_function
                y_scores = pipeline.decision_function(X_test_dense)
                y_pred_proba = np.vstack([1 - y_scores, y_scores]).T

            # Calculate metrics for each threshold
            for threshold in thresholds:
                y_pred = (y_pred_proba[:, 1] >= threshold).astype(int)

                # Calculate metrics for this threshold
                f1 = f1_score(y_test, y_pred, zero_division=1)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=1)
                recall = recall_score(y_test, y_pred, zero_division=1)
                mcc = matthews_corrcoef(y_test, y_pred)

                metrics_per_combination.append({
                    'dt__criterion': param_dict['dt__criterion'],
                    'dt__max_depth': param_dict['dt__max_depth'],
                    'dt__min_samples_split': param_dict['dt__min_samples_split'],
                    'dt__min_samples_leaf': param_dict['dt__min_samples_leaf'],
                    'dt__max_features': param_dict['dt__max_features'],
                    'fold': fold + 1,
                    'threshold': threshold,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                })

    if len(metrics_per_combination) == 0:
        print("No valid folds. Exiting.")
        return None, None

    # Convert the list of metrics into a DataFrame
    df_results = pd.DataFrame(metrics_per_combination)

    # Save the per-fold results
    outFile = f"{combination_label}-thresholds-dt-results-per-fold.csv"
    df_results.to_csv(os.path.join(outDir, outFile), index=False)

    print(f"Per-fold Decision Tree analysis completed. Results saved to: {outFile}")

    # Compute the average metrics across folds for each combination of hyperparameters and thresholds
    grouped_metrics = df_results.groupby(
        [ 'dt__criterion', 'dt__max_depth', 'dt__min_samples_split', 'dt__min_samples_leaf', 'dt__max_features', 'threshold']
    ).agg({
        'accuracy': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'mcc': ['mean', 'std']
    }).reset_index()

    # Flatten the MultiIndex columns
    grouped_metrics.columns = [
        '_'.join(col).strip('_') if col[1] else col[0] 
        for col in grouped_metrics.columns.values
    ]

    # Save the averaged results
    outFileAvg = f"{combination_label}-thresholds-dt-results-averaged.csv"
    grouped_metrics.to_csv(os.path.join(outDir, outFileAvg), index=False)

    print(f"Averaged results saved to: {outFileAvg}")

    return df_results, grouped_metrics

if __name__ == "__main__":

        # Define the parameter grid for hyperparameter tuning
    param_grid = {
    'dt__max_depth': [10, 20, 30],                  # Maximum depth of the decision tree
    'dt__min_samples_split': [5, 10],               # Minimum number of samples required to split a node
    'dt__min_samples_leaf': [2, 5],                 # Minimum number of samples required at a leaf node
    'dt__criterion': ['gini', 'entropy'],           # Function to measure the quality of a split
    'dt__max_features': [None, 'sqrt', 'log2'],     # Controls how many features to consider for splits
    }

    # Paths to your datasets
    flakyZip = "Dataset/flaky_files.zip"
    nonFlakyZip = "Dataset/nonflaky_files.zip"  # Unbalanced dataset

    # Create result and extract directories
    outDir = "results/dt_thresholds/"
    os.makedirs(outDir, exist_ok=True)
    extractDir = "extracted/dt_thresholds/"
    os.makedirs(extractDir, exist_ok=True)

    # Perform Decision Tree analysis with threshold adjustments
    print("Starting Decision Tree analysis with threshold adjustments...")
    df_results, grouped_metrics = flastDTWithThresholds(
        outDir, flakyZip, nonFlakyZip, extractDir, n_splits=5,
        combination_label="dt_thresholds", param_grid=param_grid)

    if df_results is not None:
        print("Analysis completed. Per-fold Results:")
        print(df_results.head())

        print("Averaged Results:")
        print(grouped_metrics.head())
    else:
        print("Analysis did not produce any results.")


Starting Decision Tree analysis with threshold adjustments...
Number of flaky documents: 45
Number of non-flaky documents: 243
Total number of documents: 288
Training with parameters: {'dt__max_depth': 10, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 2, 'dt__criterion': 'gini', 'dt__max_features': None}
Training with parameters: {'dt__max_depth': 10, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 2, 'dt__criterion': 'gini', 'dt__max_features': 'sqrt'}
Training with parameters: {'dt__max_depth': 10, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 2, 'dt__criterion': 'gini', 'dt__max_features': 'log2'}
Training with parameters: {'dt__max_depth': 10, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 2, 'dt__criterion': 'entropy', 'dt__max_features': None}
Training with parameters: {'dt__max_depth': 10, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 2, 'dt__criterion': 'entropy', 'dt__max_features': 'sqrt'}
Training with parameters: {'dt__max_depth': 10, 'dt__min_sam

Training with parameters: {'dt__max_depth': 30, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 2, 'dt__criterion': 'entropy', 'dt__max_features': 'log2'}
Training with parameters: {'dt__max_depth': 30, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 5, 'dt__criterion': 'gini', 'dt__max_features': None}
Training with parameters: {'dt__max_depth': 30, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 5, 'dt__criterion': 'gini', 'dt__max_features': 'sqrt'}
Training with parameters: {'dt__max_depth': 30, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 5, 'dt__criterion': 'gini', 'dt__max_features': 'log2'}
Training with parameters: {'dt__max_depth': 30, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 5, 'dt__criterion': 'entropy', 'dt__max_features': None}
Training with parameters: {'dt__max_depth': 30, 'dt__min_samples_split': 5, 'dt__min_samples_leaf': 5, 'dt__criterion': 'entropy', 'dt__max_features': 'sqrt'}
Training with parameters: {'dt__max_depth': 30, 'dt__min_sa